## TPOT Regressor Demo

Boston Housing Price Dataset: Regression problem to Predict median value of houses in a neighborhood.

506 datapoints of 13 continuous attributes and 1 continuous target variable (Median value of owner-occupied homes in $1,000s).

<a href="http://lib.stat.cmu.edu/datasets/boston">http://lib.stat.cmu.edu/datasets/boston</a>

    class tpot.TPOTRegressor(generations=100, 
                             population_size=100,
                             offspring_size=None, 
                             mutation_rate=0.9,
                             crossover_rate=0.1,
                             scoring='neg_mean_squared_error', 
                             cv=5,
                             subsample=1.0, n_jobs=1,
                             max_time_mins=None, 
                             max_eval_time_mins=5,
                             random_state=None, 
                             config_dict=None,
                             warm_start=False, 
                             verbosity=0,
                             disable_update_check=False)

In [1]:
import numpy as np
import pandas as pd  
import time
from tpot import TPOTRegressor
from tpot import TPOTClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from xgboost import XGBRegressor
from sklearn import preprocessing
from sklearn.linear_model import RidgeCV



housing_data = pd.read_csv('housing.csv')    
data = housing_data.drop('class', axis=1).values
targets = housing_data.loc[:, 'class'].values

print("---data---")
print(data)

print("---targets---")
print(targets)

X_train, X_test, y_train, y_test = train_test_split(data, targets, train_size=0.75, test_size=0.25)

# http://scikit-learn.org/stable/modules/preprocessing.html
# Standardize to zero mean and unit variance
X_train = preprocessing.scale(X_train)
X_test = preprocessing.scale(X_test)

print("---x training: post-standardization---")
print(X_train)

---data---
[[  38.3518     0.        18.1     ...,   20.2      396.9       30.59   ]
 [  67.9208     0.        18.1     ...,   20.2      384.97      22.98   ]
 [  25.0461     0.        18.1     ...,   20.2      396.9       26.77   ]
 ..., 
 [   6.53876    0.        18.1     ...,   20.2      392.05       2.96   ]
 [   9.2323     0.        18.1     ...,   20.2      366.15       9.53   ]
 [   8.26725    0.        18.1     ...,   20.2      347.88       8.88   ]]
---targets---
[  5.    5.    5.6   6.3   7.    7.    7.2   7.2   7.2   7.4   7.5   8.1
   8.3   8.3   8.4   8.4   8.5   8.5   8.7   8.8   8.8   9.5   9.6   9.7
  10.2  10.2  10.2  10.4  10.4  10.5  10.5  10.8  10.9  10.9  11.   11.3
  11.5  11.7  11.7  11.8  11.8  11.9  11.9  12.   12.1  12.3  12.5  12.6
  12.7  12.7  12.7  12.8  13.   13.1  13.1  13.1  13.1  13.2  13.3  13.3
  13.3  13.4  13.4  13.4  13.4  13.5  13.5  13.6  13.6  13.8  13.8  13.8
  13.8  13.8  13.9  13.9  14.   14.1  14.1  14.1  14.2  14.3  14.3  14.4
  14.4  14.5

/Users/orcsy/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Take the manual approach

randomForestRegressor = RandomForestRegressor()
randomForestRegressor.fit(X_train, y_train)

linearRegression = LinearRegression()
linearRegression.fit(X_train, y_train)

mLPRegressor = MLPRegressor(hidden_layer_sizes=(10,10), max_iter=10000)
mLPRegressor.fit(X_train, y_train)

xGBRegressor = XGBRegressor(max_depth=7, min_child_weight=5, nthread=1)
xGBRegressor.fit(X_train, y_train)

adaBoostRegressor = AdaBoostRegressor()
adaBoostRegressor.fit(X_train, y_train)

gradientBoostingRegressor = GradientBoostingRegressor()
gradientBoostingRegressor.fit(X_train, y_train)

kNeighborsRegressor = KNeighborsRegressor()
kNeighborsRegressor.fit(X_train, y_train)

print("Linear Regression: {:.2f}".format(mean_absolute_error(y_test, linearRegression.predict(X_test))))
print("Neural Network: {:.2f}".format(mean_absolute_error(y_test, mLPRegressor.predict(X_test))))
print("XG Boost: {:.2f}".format(mean_absolute_error(y_test, xGBRegressor.predict(X_test))))
print("ADA Boost: {:.2f}".format(mean_absolute_error(y_test, adaBoostRegressor.predict(X_test))))
print("Gradient Boost: {:.2f}".format(mean_absolute_error(y_test, gradientBoostingRegressor.predict(X_test))))
print("KNN: {:.2f}".format(mean_absolute_error(y_test, kNeighborsRegressor.predict(X_test))))
print("Random Forest: {:.2f}".format(mean_absolute_error(y_test, randomForestRegressor.predict(X_test))))


Linear Regression: 3.65
Neural Network: 2.77
XG Boost: 2.37
ADA Boost: 2.95
Gradient Boost: 2.72
KNN: 3.10
Random Forest: 2.64


In [3]:
# Take the tpot approach
start_time = time.clock()
tpot = TPOTRegressor(generations=3, population_size=7, verbosity=2, cv=4, scoring = 'neg_mean_absolute_error')
tpot.fit(X_train, y_train) 
tpot.export('tpot_boston_pipeline.py')

end_time = time.clock()
print("--- Done in {:.2f} seconds ---".format(end_time - start_time))
print("TPot: {:.2f}".format(tpot.score(X_test, y_test)))

Optimization Progress:  50%|█████     | 14/28 [00:06<00:07,  1.78pipeline/s]

Generation 1 - Current best internal CV score: 2.4166486174212127


Optimization Progress:  75%|███████▌  | 21/28 [00:07<00:03,  2.17pipeline/s]

Generation 2 - Current best internal CV score: 2.4166486174212127


Generation 3 - Current best internal CV score: 2.4166486174212127

Best pipeline: ExtraTreesRegressor(RandomForestRegressor(input_matrix, RandomForestRegressor__bootstrap=False, RandomForestRegressor__max_features=0.4, RandomForestRegressor__min_samples_leaf=1, RandomForestRegressor__min_samples_split=20, RandomForestRegressor__n_estimators=DEFAULT), ExtraTreesRegressor__bootstrap=False, ExtraTreesRegressor__max_features=0.75, ExtraTreesRegressor__min_samples_leaf=DEFAULT, ExtraTreesRegressor__min_samples_split=9, ExtraTreesRegressor__n_estimators=100)
--- Done in 10.13 seconds ---
TPot: 2.54
